In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_colwidth', None) # para visualizar todo el contenido de los valores
#pd.set_option('display.max_rows', None) # Para visualizar todo el contenido de las filas

In [2]:
# Leemos el csv

datos = pd.read_csv('Datos/datos_empresa.csv', index_col=0)

### Transformación de Datos


In [3]:
# Borramos columnas que no necesitamos según el estudio EDA

columnas_borrar = ['employeecount','Over18','SameAsMonthlyIncome', 'Salary', 'RoleDepartament', 'NUMBERCHILDREN','employeenumber','StandardHours']

def borrar_colunas (datos, columnas):
    datos.drop( columns = columnas, inplace= True)

borrar_colunas(datos,columnas_borrar)

In [4]:
# Transformamos las cabeceras de las columnas para su mejor visulatización las convertimos todos a lower

new_columns = {column : column.lower() for column in datos}

new_columns

datos.rename(columns = new_columns, inplace= True)


#comprobamos que se ha realizado
datos.sample(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework
1510,26,No,travel_frequently,"1096,0$",NaN,6,3,Other,3,0,61,4,1,laboraTORY TechNiciaN,4,Married,NaN,7102,0,No,18,"3,0",1,1,"8,0",3,"3,0",7,NaN,7,7,1997,True
404,29,No,travel_rarely,"694,0$",NaN,1,3,NaN,4,1,Not Available,2,4,ReSeArCh dirECTOR,4,Divorced,NaN,3423,3,No,14,"3,0",2,2,"9,0",2,"2,0",7,NaN,1,7,1994,True


In [5]:
# Columna Age --- cambiamos los valores string a números

datos['age'].unique()

age = datos['age'].str.replace('fifty-eight', '58 ').str.replace('fifty-eight', '47 ').str.replace('thirty-six', '36').str.replace('fifty-five', '55').str.replace('fifty-two', '52').str.replace('thirty-one', '31').str.replace('twenty-six', '36').str.replace('thirty-seven', '37').str.replace('thirty-seven', '37').str.replace('thirty-two', '32').str.replace('twenty-four', '24').str.replace('forty-seven', '47').str.replace('thirty', '30')

datos['age'] = age

datos['age'] = datos['age']

datos['age'].unique()  



array(['51', '52', '42', '47', '46', '48', '59', '41', '56', '38', '55',
       '40', '58', '35', '45', '33', '36', '34', '53', '43', '60', '32',
       '37', '49', '39', '50', '44', '30', '58 ', '29', '31', '54', '57',
       '27', '28', '26', '25', '24', '23', '22', '21', '20', '19', '18'],
      dtype=object)

In [6]:
# Eliminamos los $ y ,0 con un .replace. También hemos pasado los nan y luego cambiamos el tipo a float

dailyrate = datos['dailyrate'].str.replace('$', '')
datos['dailyrate'] = dailyrate

datos['dailyrate'] = datos['dailyrate']

datos['dailyrate'].head(2)

0    684,0
1    699,0
Name: dailyrate, dtype: object

In [7]:
# Hemos cambiado el tipo de int a str y eliminado el menos de los negativos. Lo pasamos a int.

datos_distance = datos['distancefromhome'].astype(str)

distancefromhome = datos_distance.str.replace('-', '')

datos['distancefromhome'] = distancefromhome

datos['distancefromhome'] = datos['distancefromhome']

datos['distancefromhome'].unique()

array(['6', '1', '4', '2', '3', '22', '25', '9', '7', '23', '10', '12',
       '14', '13', '15', '8', '42', '28', '37', '5', '16', '35', '26',
       '24', '29', '17', '21', '18', '30', '27', '20', '31', '39', '11',
       '19', '33', '34', '46', '36', '45', '47', '32', '41', '49', '48',
       '38', '43', '40', '44'], dtype=object)

In [8]:
# Vamos a sustituir los 0 y 1 por la siniciales de su género y cambiarlo a tipo objeto.
dic_map = {0:'M', 1:'F'}

datos["gender"] = datos["gender"].map(dic_map)

datos['gender'].unique()

array(['M', 'F'], dtype=object)

In [9]:
# cambiar el Not Avaiable a NaN

datos['hourlyrate'] = datos['hourlyrate'].replace('Not Available', np.nan)

datos['hourlyrate'].unique() 

array(['51', '65', '58', '82', '45', '99', '91', '64', '55', '68', '49',
       '61', '79', '31', '69', '48', '80', '74', '98', '59', '33', '56',
       '66', '57', '53', '87', '81', '84', '32', '41', '92', '47', nan,
       '43', '86', '30', '42', '88', '96', '67', '62', '72', '78', '89',
       '52', '50', '90', '37', '94', '76', '60', '46', '83', '100', '40',
       '97', '54', '75', '39', '85', '63', '44', '93', '36', '35', '73',
       '71', '70', '38', '77', '95', '34'], dtype=object)

In [10]:
# Sustituimos los que están mal escritos para que sean iguales a los que si creando un diccionario previamente. Comprobar moda
replacements = {
    'divorced': 'Divorced',
    'Marreid': 'Married'
}

datos['maritalstatus'] = datos['maritalstatus'].replace(replacements)
datos['maritalstatus'].unique() 

array([nan, 'Married', 'Divorced', 'Single'], dtype=object)

In [11]:
# Cambiamos todos los datos a yes o NO depende la numeración

diccionario_mapa = {'Yes':'Yes', '1':'Yes', 'False':'No', '0':'No', 'True':'Yes'}

datos["remotework"] = datos["remotework"].map(diccionario_mapa)

datos['remotework'].unique()

array(['Yes', 'No'], dtype=object)

In [12]:
# Funcion para cambiar a float

cols_float = ['dailyrate', 'monthlyincome',  'performancerating', 'totalworkingyears', 'worklifebalance', 'yearsincurrentrole']

def cambiar_float(datos, cols):
    for col in cols:
        datos[col] = datos[col].apply(lambda dato: float(dato.replace(",", ".")) if isinstance(dato, str) else np.nan)

cambiar_float(datos, cols_float)

In [13]:
# Funcion para cambiar a int

cols_int = ['age', 'distancefromhome', 'hourlyrate']

def cambiar_float(datos, cols):
    for col in cols:
        datos[col] = datos[col].apply(lambda dato: int(dato) if isinstance(dato, str) else np.nan)

cambiar_float(datos, cols_int)

In [14]:
# Funcion para cambiar textos

cols_text = ['attrition', 'businesstravel', 'department', 'educationfield', 'gender', 'jobrole', 'maritalstatus', 'overtime', 'remotework']

def cambiar_texto(datos, cols):
    for col in cols:
        datos[col] = datos[col].str.strip().str.replace('-', ' ').str.replace('_', ' ').str.capitalize()
        

cambiar_texto(datos, cols_text)

In [15]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1614 entries, 0 to 1613
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       1614 non-null   int64  
 1   attrition                 1614 non-null   object 
 2   businesstravel            842 non-null    object 
 3   dailyrate                 1490 non-null   float64
 4   department                302 non-null    object 
 5   distancefromhome          1614 non-null   int64  
 6   education                 1614 non-null   int64  
 7   educationfield            869 non-null    object 
 8   environmentsatisfaction   1614 non-null   int64  
 9   gender                    1614 non-null   object 
 10  hourlyrate                1530 non-null   float64
 11  jobinvolvement            1614 non-null   int64  
 12  joblevel                  1614 non-null   int64  
 13  jobrole                   1614 non-null   object 
 14  jobsatisfacti

In [16]:
datos.sample(5)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework
360,44,No,NaN,661.0,NaN,9,2,NaN,2,M,61.0,3,1,Research scientist,1,NaN,NaN,7508,1,Yes,13,3.0,4,0,8.0,0,3.0,8,NaN,7,1,1979,Yes
771,39,No,NaN,1253.0,Research & development,10,1,Medical,3,M,65.0,3,3,Research director,3,Single,NaN,7914,7,NaN,21,4.0,3,0,NaN,3,3.0,4,NaN,2,2,1984,Yes
258,39,No,Travel rarely,119.0,NaN,15,4,Marketing,2,M,77.0,3,4,Sales executive,1,Single,13341.0,25098,0,No,12,3.0,1,0,21.0,3,3.0,20,NaN,11,10,1984,No
1519,37,No,NaN,1192.0,NaN,5,2,Medical,4,M,61.0,3,2,Manufacturing director,4,NaN,6347.0,23177,7,No,16,3.0,3,2,8.0,2,2.0,6,NaN,0,4,1986,No
1272,29,No,Travel rarely,1210.0,NaN,2,3,Medical,1,M,78.0,2,2,Sales executive,2,Married,6644.0,3687,2,No,19,3.0,2,2,10.0,2,NaN,0,NaN,0,0,1994,Yes


In [17]:
# Creamos copia de seguridad del archivo

datos.to_csv('Datos/datos_empresa_V.1.clean.csv')

### Gestion de Nulos

In [18]:
datos = pd.read_csv('Datos/datos_empresa_V.1.clean.csv', index_col=0)
datos.sample(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,datebirth,remotework
1336,58,Yes,Travel frequently,781.0,NaN,2,1,NaN,4,M,57.0,2,1,Laboratory technician,4,Divorced,2380.0,13384,9,NaN,14,3.0,4,1,3.0,3,2.0,1,NaN,0,0,1965,Yes
780,35,No,Travel rarely,682.0,NaN,18,4,NaN,2,M,71.0,3,2,Sales executive,1,NaN,5561.0,15975,0,NaN,16,3.0,4,1,NaN,2,1.0,5,NaN,0,4,1988,Yes


In [19]:
# qué columnas categóricas tienen nulos

nulos_cat = datos[datos.columns[datos.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['businesstravel', 'department', 'educationfield', 'maritalstatus',
       'overtime'],
      dtype='object')


In [20]:
# qué columnas numericas tienen nulos

nulos_num = datos[datos.columns[datos.isnull().any()]].select_dtypes(include = ['int', 'float']).columns
print("Las columnas numéricas que tienen nulos son : \n ")
print(nulos_num)

Las columnas numéricas que tienen nulos son : 
 
Index(['dailyrate', 'hourlyrate', 'monthlyincome', 'performancerating',
       'totalworkingyears', 'worklifebalance', 'yearsincurrentrole'],
      dtype='object')


In [21]:
# Vemos la forma de estas columnas categóricas

for col in nulos_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(datos[col].value_counts() / datos.shape[0])
    print("........................")

La distribución de las categorías para la columna BUSINESSTRAVEL


businesstravel
Travel rarely        0.363073
Travel frequently    0.102230
Non travel           0.056382
Name: count, dtype: float64

........................
La distribución de las categorías para la columna DEPARTMENT


department
Research & development    0.121437
Sales                     0.056382
Human resources           0.009294
Name: count, dtype: float64

........................
La distribución de las categorías para la columna EDUCATIONFIELD


...

........................
La distribución de las categorías para la columna MARITALSTATUS


maritalstatus
Married     0.271995
Single      0.201363
Divorced    0.123296
Name: count, dtype: float64

........................
La distribución de las categorías para la columna OVERTIME


overtime
No     0.422553
Yes    0.158612
Name: count, dtype: float64

........................


In [22]:
# Vemos la forma de estas columnas numéricas

for col in nulos_num:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(datos[col].value_counts() / datos.shape[0])
    print("........................")

La distribución de las categorías para la columna DAILYRATE


dailyrate
691.0    0.004337
329.0    0.004337
147.0    0.003717
408.0    0.003717
530.0    0.003717
           ...   
317.0    0.000620
891.0    0.000620
759.0    0.000620
483.0    0.000620
105.0    0.000620
Name: count, Length: 848, dtype: float64

........................
La distribución de las categorías para la columna HOURLYRATE


hourlyrate
42.0    0.020446
66.0    0.019827
48.0    0.018587
84.0    0.017968
57.0    0.017968
          ...   
47.0    0.009294
68.0    0.008055
53.0    0.008055
38.0    0.007435
34.0    0.006815
Name: count, Length: 71, dtype: float64

........................
La distribución de las categorías para la columna MONTHLYINCOME


...

........................
La distribución de las categorías para la columna WORKLIFEBALANCE


worklifebalance
3.0    0.565675
2.0    0.222429
4.0    0.096035
1.0    0.048947
Name: count, dtype: float64

........................
La distribución de las categorías para la columna YEARSINCURRENTROLE


yearsincurrentrole
2.0     0.006815
7.0     0.003098
0.0     0.002478
4.0     0.001859
1.0     0.001859
11.0    0.001239
6.0     0.001239
3.0     0.001239
13.0    0.000620
12.0    0.000620
Name: count, dtype: float64

........................


In [23]:
# porcentajes de todas

datos.isna().sum() / datos.shape[0] * 100

age                          0.000000
attrition                    0.000000
businesstravel              47.831475
dailyrate                    7.682776
department                  81.288724
distancefromhome             0.000000
education                    0.000000
educationfield              46.158612
environmentsatisfaction      0.000000
gender                       0.000000
hourlyrate                   5.204461
jobinvolvement               0.000000
joblevel                     0.000000
jobrole                      0.000000
jobsatisfaction              0.000000
maritalstatus               40.334572
monthlyincome               52.230483
monthlyrate                  0.000000
numcompaniesworked           0.000000
overtime                    41.883519
percentsalaryhike            0.000000
performancerating           12.081784
relationshipsatisfaction     0.000000
stockoptionlevel             0.000000
totalworkingyears           32.589839
trainingtimeslastyear        0.000000
worklifebala

### Valores nulos totales en porcentajes

0% - 5% ---> Rellenar los valores con estadisticas (moda, media, mediana)
5% - 20% ---> O rellenar o predecir con valores de otra columna.
20% - 50% ---> Evaluar si es necesario eliminar la columna o crucial insertar datos.
50% ---> Eliminar dependiendo del tipo de dato.

- businesstravel, la cual tiene 47.831475% nulos. Calcularemos el porcentaje de cada categoría para determinar la moda porque no es numérica. Pero no eliminaremos porque la consideramos importante.

- dailyrate, la cual tiene 7.682776% nulos. Calcular valores estadísticos.

- department , la cual tiene 81.288724% nulos. Creemos que si tiene información clave para nuestros datos. Puede ser gente que no haya querido responder en la encuesta. Le daremos una nueva categoría "Uknown"               

- monthlyincome, la cual tiene 52.230483% nulos. Sustituimos los nan por 0 como categoría que desconocemos.

- educationfield, la cual tiene 46.158612% nulos. La consideramos irrelevante, la eliminaremos.

- hourlyrate, la cual tiene 5.204461% nulos. Calcular valores estadísticos.

- maritalstatus, la cual tiene 40.334572% nulos. Crearemos otra categoría "Others"

- overtime, la cual tienne 41.883519% nulos. Miraremos la proporción de % y tiraremos por la mayoría.

- performancerating , la cual tiene 41.883519% nulos. Crearemos nueva categoría "0" que es sin información.

- totalworkingyears, la cual tiene 32.589839% nulos. Crearemos nueva categoría "0" que es sin información.

- worklifebalance, la cual tiene 6.691450% nulos. Calcular valores estadísticos.

- yearsincurrentrole, la cual tiene 97.893432% nulos. Creemos que da información similar a 'yearssincelastpromotion'. La quitaremos.

In [24]:
# Borramos las columnas que tras su analisis de nulos hemos determinado que se van fuera.

columnas_borrar2 = ['educationfield','yearsincurrentrole', 'monthlyincome']

borrar_colunas(datos,columnas_borrar2)

In [25]:
# Columnas con las que nos quedamos finalmente hasta este punto.

datos.columns

Index(['age', 'attrition', 'businesstravel', 'dailyrate', 'department',
       'distancefromhome', 'education', 'environmentsatisfaction', 'gender',
       'hourlyrate', 'jobinvolvement', 'joblevel', 'jobrole',
       'jobsatisfaction', 'maritalstatus', 'monthlyincome', 'monthlyrate',
       'numcompaniesworked', 'overtime', 'percentsalaryhike',
       'performancerating', 'relationshipsatisfaction', 'stockoptionlevel',
       'totalworkingyears', 'trainingtimeslastyear', 'worklifebalance',
       'yearsatcompany', 'yearssincelastpromotion', 'yearswithcurrmanager',
       'datebirth', 'remotework'],
      dtype='object')

In [26]:
# businesstravel, la cual tiene 47.831475% nulos. Calcularemos el porcentaje de cada categoría para determinar la moda porque no es numérica. Pero no eliminaremos porque la consideramos importante.

datos['businesstravel'].value_counts(normalize=True) * 100

businesstravel
Travel rarely        69.596200
Travel frequently    19.596200
Non travel           10.807601
Name: proportion, dtype: float64

In [27]:
# Imputamos la columna businesstravel

datos['businesstravel'].replace('NaN', np.nan, inplace=True)

datos['businesstravel'].fillna('Travel rarely', inplace=True)

datos['businesstravel'].unique()

array(['Travel rarely', 'Travel frequently', 'Non travel'], dtype=object)

In [28]:
# Calcular los valores estadísticos de 'dailyrate'ArithmeticError

datos['dailyrate'].describe()

count    1490.000000
mean      802.085235
std       403.246954
min       103.000000
25%       468.250000
50%       798.000000
75%      1157.000000
max      1499.000000
Name: dailyrate, dtype: float64

In [29]:
# Viendo los datos, la media y mediana tienen poca diferencia. Así que vamos a sacar la mediana y sustituirlo por ella.

datos['dailyrate'].fillna(datos['dailyrate'].median(), inplace=True)
datos['dailyrate'].isna().sum() # Para visualizar que no quedan nulos

0

In [30]:
datos['department'].unique

<bound method Series.unique of 0                          NaN
1                          NaN
2       Research & development
3                          NaN
4                          NaN
                 ...          
1609                       NaN
1610                       NaN
1611                       NaN
1612                       NaN
1613                       NaN
Name: department, Length: 1614, dtype: object>

In [31]:
# Vamos a crear una categoría nueva 'Uknown' para la columna 'department'

datos['department'].replace('NaN', np.nan, inplace=True)
datos['department'].fillna('Unknown', inplace=True)
datos['department'].unique()

array(['Unknown', 'Research & development', 'Sales', 'Human resources'],
      dtype=object)

In [32]:
# hourlyrate, calcular valores estadísticos.

datos['hourlyrate'].describe()

count    1530.000000
mean       66.028105
std        20.185954
min        30.000000
25%        48.000000
50%        66.000000
75%        84.000000
max       100.000000
Name: hourlyrate, dtype: float64

In [33]:
# Nos volvemos a quedar con la mediana al tener apenas diferencia con la media.

datos['hourlyrate'].fillna(datos['hourlyrate'].median(), inplace=True)
datos['hourlyrate'].isna().sum() 

0

In [34]:
# maritalstatus, crearemos otra categoría "Others"

datos['maritalstatus'].replace('NaN', np.nan, inplace=True)
datos['maritalstatus'].fillna('Other', inplace=True)
datos['maritalstatus'].unique()

array(['Other', 'Married', 'Divorced', 'Single'], dtype=object)

In [35]:
# overtime, miraremos la proporción de % y tiraremos por la mayoría.

datos['overtime'].value_counts(normalize=True) * 100

overtime
No     72.707889
Yes    27.292111
Name: proportion, dtype: float64

In [36]:
datos['overtime'].replace('NaN', np.nan, inplace=True)
datos['overtime'].fillna('No', inplace=True)
datos['overtime'].unique()

array(['No', 'Yes'], dtype=object)

In [37]:
# performancerating, crearemos nueva categoría "0" que es sin información.

datos['performancerating'].fillna(0, inplace=True)
datos['performancerating'].unique()


array([3., 4., 0.])

In [41]:
# totalworkingyears, la cual tiene 32.589839% nulos. Vamos a hacerle la mediana/moda.

datos['totalworkingyears'].describe()

count    1088.000000
mean       11.318934
std         7.727675
min         0.000000
25%         6.000000
50%        10.000000
75%        15.000000
max        40.000000
Name: totalworkingyears, dtype: float64

In [43]:
datos['totalworkingyears'].fillna(datos['totalworkingyears'].median(), inplace=True)
datos['totalworkingyears'].isna().sum() 

0

In [44]:
# worklifebalance, la cual tiene 6.691450% nulos. Calcular valores estadísticos.

datos['worklifebalance'].describe()


count    1506.000000
mean        2.759628
std         0.702622
min         1.000000
25%         2.000000
50%         3.000000
75%         3.000000
max         4.000000
Name: worklifebalance, dtype: float64

In [45]:
# Nos quedamos con la mediana

datos['worklifebalance'].fillna(datos['worklifebalance'].median(), inplace=True)
datos['worklifebalance'].isna().sum() 

0

In [48]:
# monthlyincome, la cual tiene 52.230483% nulos. Sustituimos los nan por 0 como categoría que desconocemos.

datos['monthlyincome'].fillna(0, inplace=True)
datos['monthlyincome'].unique()

array([19537., 19999., 19232., 17169.,     0., 17174., 16595., 19973.,
       13402., 13206., 19545., 18041., 19246., 10748., 16752.,  6201.,
       19845.,  4001., 10447., 16064.,  3210., 10266., 10475.,  6162.,
        4721.,  4615., 16959., 10306.,  5406.,  5902., 10855.,  5914.,
        6646., 13973., 13320.,  6687.,  4735., 13872., 19045., 16015.,
        9613., 11510.,  4306., 17046.,  5067.,  3692., 19847.,  2308.,
        5747., 10422.,  6347.,  2348.,  3072., 12490.,  8020., 17068.,
        8943., 19272.,  5577.,  2691.,  7403.,  8823.,  3579., 17779.,
       18213., 13577., 19190., 17123., 19187., 10008.,  7988.,  7083.,
        4723.,  3407.,  2929., 12031., 15427.,  5126.,  9619.,  5010.,
       19033., 10400.,  2793.,  5674., 19197.,  8412.,  3348.,  5309.,
        7654., 12808.,  6586.,  2519., 17099., 13120.,  3131.,  4444.,
        6674.,  9852., 17861.,  7446., 13757., 16598.,  4285., 10124.,
        9714.,  9924., 11245.,  3069.,  9241.,  6472.,  3055.,  8321.,
      

In [49]:
# Creamos el archivo definitivo sin nulos

datos.to_csv('Datos/datos_empresa_nonulos.csv')